In [253]:
import pandas as pd
import random
import spacy
from spacy import displacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en')


# Ingreso de los simtomas detectados por el terapista

In [254]:
#texto = "A clinical and histological study of Ullrich's disease (congenital atonic-sclerotic muscular dystrophy"  
texto = input("Ingrese la consulta: ")

Ingrese la consulta: Nutritional status evaluation in patients affected by Bethlem Myopathy and Ullrich Congenital Muscular Dystrophy


# Metodo categorizador

In [255]:
doc = nlp(texto)
importante=[]
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,token.shape_, token.is_alpha, token.is_stop)
    if token.pos_=='INTJ' or token.pos_=="ADJ" or token.pos_=="NOUN" or token.pos_=="ADV":
        importante.append(token.text)
#Funcion que permite ordenar el list en forma del abecedario.
importante.sort()
print("Palabras a buscar",importante)

Nutritional nutritional ADJ JJ amod Xxxxx True False
status status NOUN NN compound xxxx True False
evaluation evaluation NOUN NN ROOT xxxx True False
in in ADP IN prep xx True True
patients patient NOUN NNS pobj xxxx True False
affected affect VERB VBN acl xxxx True False
by by ADP IN agent xx True True
Bethlem Bethlem PROPN NNP compound Xxxxx True False
Myopathy Myopathy PROPN NNP pobj Xxxxx True False
and and CCONJ CC cc xxx True True
Ullrich Ullrich PROPN NNP compound Xxxxx True False
Congenital Congenital PROPN NNP compound Xxxxx True False
Muscular Muscular PROPN NNP compound Xxxxx True False
Dystrophy Dystrophy PROPN NNP conj Xxxxx True False
Palabras a buscar ['Nutritional', 'evaluation', 'patients', 'status']


# Definimos Peso 
formula:  1/tamano del token. 

In [256]:
peso=1/len(importante)
print(peso)

0.25


Este valor nos permite definir el peso de cada categoria titulo. Este valor nos permite categorizar entre la importancia que tiene cada uno.

# Consulta Por Categoria Titulo en Neo4j

In [268]:
#Recorrido dentro de la base 
articulos_obtenidos=[]
aux_diccionario={}
for i in range (len(importante)):
    print(importante[i])
    try:
        coincidencia=graph.run('match(n:palabraclave{nombre:"'+importante[i]+'"})<-[:rdfs__subClassOf]-(u:categoriaTitulo)<-[:rdfs__subClassOf]-(i:instancia)-[:rdfs__subClassOf]->(t:title) return n,u,i,id(t),t').to_data_frame()
        print("Articulos que contienen:",importante[i])
        print(list(coincidencia.get("id(t)")))
        aux_pesos=list(coincidencia.get("id(t)"))
        #Este for permite guardar todos lo id de los articulos relacionados con esa palabra
        for i in range(len(aux_pesos)):
            articulos_obtenidos.append(list(coincidencia.get("id(t)"))[i])
            #print(list(coincidencia.get("t"))[i])
            aux_diccionario[list(coincidencia.get("id(t)"))[i]]=list(coincidencia.get("t"))[i]
        print("-------------------------")
    except:
        print("no se encontro en la base la palabra ",importante[i])
    

Nutritional
Articulos que contienen: Nutritional
[177100, 197791, 198039, 200108, 214576]
-------------------------
evaluation
Articulos que contienen: evaluation
[177100, 180869, 182977, 184798, 185750, 186841, 187474, 188652, 190163, 191326, 192281, 193117, 193868, 194342, 194654, 195286, 195670, 202260, 202375, 205456, 205520, 206024, 206217, 206425, 206665, 207407, 207615, 209207, 209992, 211452, 213651, 214274, 214372, 214797, 214837, 218156, 219013, 219535, 220030, 221810, 223129, 224341, 224901, 227248, 228796, 230459, 231342, 234417, 234674, 235028, 235287, 236574, 236782, 237406, 240221, 240915, 247533, 248187, 249869, 249885, 250612, 254863, 256094, 256558, 256909, 257189, 260521]
-------------------------
patients
Articulos que contienen: patients
[177100, 178297, 178598, 178677, 178858, 179950, 180091, 180124, 180132, 180211, 180319, 180342, 180542, 180869, 180973, 181029, 181487, 181591, 181599, 182969, 183291, 183400, 183726, 183814, 183942, 183950, 184030, 184054, 184134

# Generacion de recomendacion por pesos

In [269]:
#primero eliminar los repetidos 
limpieza=list(set(articulos_obtenidos))

#despues contar la cantidad de veces que se repite un id en general.
diccionario_recomendacion={}
for i in range (len(limpieza)):
    print('Del id ',limpieza[i],'se encontraron :',articulos_obtenidos.count(limpieza[i]),"este articulo tiene un peso de:",peso*articulos_obtenidos.count(limpieza[i]))
    diccionario_recomendacion[limpieza[i]]=peso*articulos_obtenidos.count(limpieza[i])


Del id  227328 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  253955 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  217092 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  204808 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  227336 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  249869 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  235535 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  237585 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  225300 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  200727 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  249885 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  210974 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  213021 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  215071 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id

Del id  232229 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  238380 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  187190 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  238396 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  230211 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  228164 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  248648 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  242506 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  217932 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  215887 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  185170 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  213851 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  240475 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  191326 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id

Del id  255236 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  224525 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  234769 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  199954 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  240915 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  236822 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  255259 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  249118 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  224549 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  220454 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  230699 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  197935 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  255283 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  197951 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id

Del id  255990 se encontraron : 1 este articulo tiene un peso de: 0.25
Del id  231422 se encontraron : 1 este articulo tiene un peso de: 0.25


# Ordenar List De Mayor a Menor

In [275]:
import operator
clients_sort = sorted(diccionario_recomendacion.items(), key=operator.itemgetter(1), reverse=True)
for name in enumerate(clients_sort):
    #print(' Titulo del articulo:',aux_diccionario.get(name[1][0]))
    print(name[1][0], 'con una relacion del ', diccionario_recomendacion[name[1][0]])
    

177100 con una relacion del  1.0
197791 con una relacion del  0.75
221348 con una relacion del  0.5
223440 con una relacion del  0.5
248187 con una relacion del  0.5
180869 con una relacion del  0.5
213651 con una relacion del  0.5
254863 con una relacion del  0.5
234417 con una relacion del  0.5
209992 con una relacion del  0.5
214274 con una relacion del  0.5
198039 con una relacion del  0.5
214467 con una relacion del  0.5
210390 con una relacion del  0.5
235028 con una relacion del  0.5
214576 con una relacion del  0.5
257655 con una relacion del  0.5
224901 con una relacion del  0.5
210670 con una relacion del  0.5
219013 con una relacion del  0.5
227328 con una relacion del  0.25
253955 con una relacion del  0.25
217092 con una relacion del  0.25
204808 con una relacion del  0.25
227336 con una relacion del  0.25
249869 con una relacion del  0.25
235535 con una relacion del  0.25
237585 con una relacion del  0.25
225300 con una relacion del  0.25
200727 con una relacion del  0.25

240475 con una relacion del  0.25
191326 con una relacion del  0.25
209760 con una relacion del  0.25
252779 con una relacion del  0.25
209776 con una relacion del  0.25
224114 con una relacion del  0.25
213875 con una relacion del  0.25
217972 con una relacion del  0.25
207735 con una relacion del  0.25
220030 con una relacion del  0.25
213891 con una relacion del  0.25
232325 con una relacion del  0.25
248711 con una relacion del  0.25
244616 con una relacion del  0.25
252811 con una relacion del  0.25
256909 con una relacion del  0.25
224146 con una relacion del  0.25
213907 con una relacion del  0.25
218004 con una relacion del  0.25
185240 con una relacion del  0.25
234393 con una relacion del  0.25
207783 con una relacion del  0.25
185256 con una relacion del  0.25
197544 con una relacion del  0.25
209840 con una relacion del  0.25
207807 con una relacion del  0.25
197568 con una relacion del  0.25
211906 con una relacion del  0.25
240579 con una relacion del  0.25
209880 con una

In [271]:
print(aux_diccionario[177100])

(_177100:title {title: 'Nutritional status evaluation in patients affected by Bethlem Myopathy and Ullrich Congenital Muscular Dystrophy'})
